## Imports

In [2]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [3]:
import os
import numpy as np

## Constants

In [25]:
X_TRAIN_PATH = './MNIST/digit_xtrain.csv'
X_TEST_PATH = './MNIST/digit_xtest.csv'
Y_TRAIN_PATH = './MNIST/digit_ytrain.csv'
Y_TEST_PATH = './MNIST/digit_ytest.csv'

NUMBER_OF_CLASSES = 10
VALIDATION_SIZE = 10000

## Getting the data


In [6]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: total: 141 ms
Wall time: 157 ms


In [7]:
y_train_all.shape

(60000,)

In [8]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype = int)

In [9]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: total: 16.5 s
Wall time: 16.7 s


In [13]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype = int)

CPU times: total: 2.8 s
Wall time: 2.8 s


## Data Exploration

In [14]:
x_train_all.shape

(60000, 784)

In [15]:
#0 means that the pixel is completely white whereas the value 255 indicates that the pixel is extremely dark
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [16]:
y_train_all.shape

(60000,)

In [17]:
x_test.shape

(10000, 784)

## Data Preprocessing

In [18]:
#Re-scaling
x_train_all, x_test = x_train_all/255.0, x_test/255.0

### Convert target values to one-hot encoding

In [20]:
y_train_all = np.eye(NUMBER_OF_CLASSES)[y_train_all]

In [22]:
y_train_all.shape

(60000, 10)

In [23]:
y_test = np.eye(NUMBER_OF_CLASSES)[y_test]

In [24]:
y_test.shape

(10000, 10)

## Create validation dataset from training dataset

In [26]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]

In [27]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [28]:
x_train.shape

(50000, 784)